In [1]:
import numpy as np
import pandas as pd
import requests
import io
import requests
from contextlib import closing
import csv


In [2]:
eia = pd.read_csv("./EIA_Plant_List.csv")
epa = pd.read_csv("./EPA_plant_List.csv")
walk = pd.read_csv("./TX_crosswalk.csv")

### EIA Matches

In [3]:
print("Number of EIA Matches: ", len(eia.loc[eia["match"] == True]))
print("Number of EIA Non-Matches: ", len(eia.loc[eia["match"] == False]))

print("Percent of EIA Matches: ", round(len(eia.loc[eia["match"] == True])/len(eia), 3))
print("Percent of EIA Non-Matches: ", round(len(eia.loc[eia["match"] == False])/len(eia), 3))

Number of EIA Matches:  429
Number of EIA Non-Matches:  707
Percent of EIA Matches:  0.378
Percent of EIA Non-Matches:  0.622


### EPA Matches

In [4]:
print("Number of EPA Matches: ", len(epa.loc[epa["match"] == True]))
print("Number of EPA Non-Matches: ", len(epa.loc[epa["match"] == False]))

print("Percent of EPA Matches: ", round(len(epa.loc[epa["match"] == True])/len(epa), 3))
print("Percent of EPA Non-Matches: ", round(len(epa.loc[epa["match"] == False])/len(epa), 3))

Number of EPA Matches:  414
Number of EPA Non-Matches:  100
Percent of EPA Matches:  0.805
Percent of EPA Non-Matches:  0.195


### EIA Fuel Breakdown

In [5]:
print("\nPercent Fuel Types Matched\n")
print(eia.loc[eia["match"] == True]["Energy Source 1"].value_counts(normalize=True).round(3))
print("\nPercent Fuel Types Not Matched\n")
print(eia.loc[eia["match"] == False]["Energy Source 1"].value_counts(normalize=True).round(3))


Percent Fuel Types Matched

Natural Gas    0.928
Coal           0.072
Name: Energy Source 1, dtype: float64

Percent Fuel Types Not Matched

Natural Gas    0.372
Wind           0.281
Solar          0.115
Renewable      0.072
Hydro          0.071
Petroleum      0.042
Other          0.041
Nuclear        0.006
Name: Energy Source 1, dtype: float64


### EPA Fuel Breakdown

In [6]:
print("\nPercent Fuel Types Matched\n")
print(epa.loc[epa["match"] == True]["fuel"].value_counts(normalize=True).round(3))
print("\nPercent Fuel Types Not Matched\n")
print(epa.loc[epa["match"] == False]["fuel"].value_counts(normalize=True).round(3))


Percent Fuel Types Matched

Pipeline Natural Gas    0.874
Coal                    0.101
Natural Gas             0.012
NONE                    0.007
Other Gas               0.005
Name: fuel, dtype: float64

Percent Fuel Types Not Matched

Pipeline Natural Gas    0.92
NONE                    0.06
Petroleum Coke          0.01
Wood                    0.01
Name: fuel, dtype: float64


In [7]:
class EIA():
    def __init__(self, key=None):
        if key is None:
            print("You need a key in order to use this API.")
        else:
            self.key = key

    def electric_plant_all(self, plant_id):
        series_id = 'ELEC.PLANT.GEN.{}-ALL-ALL.Q'.format(plant_id)
        req = requests.get("https://api.eia.gov/series/?series_id={}&api_key={}".format(series_id, self.key))
        return pd.DataFrame.from_dict(req.json().get('series')[0].get('data'))

In [8]:
class EPA():
    def __init__(self, key=None):
        if key is None:
            print("You need a key in order to use this API.")
        else:
            self.key = key

    def hourlyData(self, orisCode, unitID, year, quarter):
        url = "https://api.epa.gov/FACT/1.0/emissions/hourlyData/csv/{}/{}/{}/{}?api_key={}".format(
            orisCode, 
            unitID, 
            year, 
            quarter, 
            self.key)
        try:
           data = pd.read_csv(url)
           return data
        except:
            return pd.DataFrame()

### Aggregate EPA vs EIA MWh in Q1 2019

In [9]:
epa_api = EPA("9ndbfvcHxIEgQ8KnDGhmVFdw3xiyOgqhhwdJg5Wo")
eia_api = EIA("9ndbfvcHxIEgQ8KnDGhmVFdw3xiyOgqhhwdJg5Wo")

In [10]:
epa_true = epa.loc[epa["match"] == True]
eia_true = eia.loc[eia["match"] == True]

In [11]:
walk = walk[["CAMD_PLANT_ID", "CAMD_UNIT_ID", "EIA_PLANT_ID", "EIA_GENERATOR_ID"]]
walk["EIA_PLANT_ID"] = walk["EIA_PLANT_ID"].astype(int)
# merged = eia_true.merge(walk, how="right", left_on=["Plant Code", "Generator ID"], right_on=["EIA_PLANT_ID", "EIA_GENERATOR_ID"])
# epa_true.merge(merged, how="right", left_on=["orisCode", "unitId"], right_on=["CAMD_PLANT_ID", "CAMD_UNIT_ID"]).dropna().drop_duplicates()


In [12]:
def get_eia_2019Q1(api,plant_id):
    temp = api.electric_plant_all(9)
    return temp.loc[temp[0] == "2019Q1"].reset_index(drop=True).at[0,1]

def get_EPA_2019Q1(api, plant_id, unit_id):
    return api.hourlyData(plant_id, unit_id, "2019", 1)


In [13]:
dic = {"plant_id":[], "eia":[], "epa":[], "valid":[]}
for i in walk["EIA_PLANT_ID"].unique():
    data = walk.loc[walk["CAMD_PLANT_ID"] == i]

    epa_sum = 0
    for j in data["CAMD_UNIT_ID"].unique():
        print(i, " : ", j)
        req = epa_api.hourlyData(i,j, "2019", 1)
        if req.empty:
            dic["plant_id"].append(i)
            dic["eia"].append("-1")
            dic["epa"].append("-1")
            dic["valid"].append(False)
            break
        epa_sum += epa_api.hourlyData(i,j, "2019", 1)["HourLoad"].sum()
    print(epa_sum)
    try:
        temp = eia_api.electric_plant_all(i)
        eia_sum = temp.loc[temp[0] == "2019Q1"].reset_index(drop=True).at[0,1]
    except: continue
    dic["plant_id"].append(i)
    dic["eia"].append(eia_sum)
    dic["epa"].append(epa_sum)
    dic["valid"].append(True)

yes = pd.DataFrame().from_dict(dic)


9  :  CTG-1
6290.0
127  :  1
962455.0
298  :  LIM1
298  :  LIM2
2054725.0
3439  :  4
3439  :  5
20660.0
3441  :  8
3441  :  9
265696.0
3443  :  9
134993.0
3452  :  1
3452  :  2
5866.0
3453  :  6
3453  :  7
3453  :  8
9295.0
3456  :  **4
3456  :  **5
3456  :  1
3456  :  2
3456  :  3
3456  :  GT-6A
3456  :  GT-6B
632405.0
3457  :  1
3457  :  2
424828.0
3459  :  1
3459  :  2
52242.0
3460  :  CBY1
3460  :  CBY2
27139.0
3464  :  GBY5
0
3468  :  SRB1
3468  :  SRB2
3468  :  SRB3
0.0
3469  :  THW31
3469  :  THW32
3469  :  THW33
3469  :  THW34
3469  :  THW41
3469  :  THW42
3469  :  THW43
3469  :  THW44
3469  :  THW51
3469  :  THW52
3469  :  THW53
3469  :  THW54
3469  :  THW55
3469  :  THW56
90224.0
3470  :  WAP1
3470  :  WAP2
3470  :  WAP3
3470  :  WAP4
3470  :  WAP5
3470  :  WAP6
3470  :  WAP7
3470  :  WAP8
2931231.0
3476  :  2
3476  :  3
3476  :  4
1677.0
3477  :  1
0.0
3478  :  1
3478  :  2
3478  :  3
113712.0
3482  :  151B
3482  :  152B
3482  :  153T
3482  :  154T
416774.0
3483  :  3
0
3484

In [ ]:
epa_api.hourlyData(3464,"GBY5", "2019", 1)

""


In [ ]:
dic = {"year":[], "Q":[], "to_string":[]}
year = 2015
while (year <= 2021):
    for i in range(1,5):
        dic["year"].append(year)
        dic["Q"].append(i)
        dic["to_string"].append("{}Q{}".format(year,i))
    year += 1

date = pd.DataFrame().from_dict(dic)
date.to_csv("date.csv", index = False)